## Mask R-CNN - Inspect Weights of a Trained Model

This notebook includes code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [ ]:
!pip install tensorflow==1.15.0
!pip install keras==2.1.5 --force-reinstall --no-deps --no-cache-dir

     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 512kB 47.3MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 3.8MB 38.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=9bf5b0b7ebdfcdcefceb3a48aece74cd8d01489839339d39f8a70278feb942b9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0


In [2]:
##to solve AttributeError: module 'tensorflow_core.compat.v2' has no attribute '__internal__'
!pip uninstall keras-nightly
!pip install h5py==2.10.0  

Uninstalling keras-nightly-2.5.0.dev2021032900:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras/*
    /usr/local/lib/python3.7/dist-packages/keras_nightly-2.5.0.dev2021032900.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/keras/applications/resnet50.py
    /usr/local/lib/python3.7/dist-packages/keras/backend/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/backend/cntk_backend.py
    /usr/local/lib/python3.7/dist-packages/keras/backend/common.py
    /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py
    /usr/local/lib/python3.7/dist-packages/keras/backend/theano_backend.py
    /usr/local/lib/python3.7/dist-packages/keras/engine/topology.py
    /usr/local/lib/python3.7/dist-packages/keras/initializers.py
    /usr/local/lib/python3.7/dist-packages/keras/legacy/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/legacy/interfaces.py
    /usr/local/lib/python3.7/dist-packages

In [3]:
##Error: ModuleNotFoundError: No module named 'mrcnn'
%cd /content/drive/MyDrive/Research/Thesis/Training/Project/main/Mask_RCNN-TF2
!python setup.py install

[Errno 2] No such file or directory: '/content/drive/MyDrive/Research/Thesis/Training/Project/main/Mask_RCNN-TF2'
/content
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
##ModuleNotFoundError: No module named 'keras.utils.generic_utils'
!pip list | grep tf
!pip install 'h5py==2.10.0' --force-reinstall

  Using cached https://files.pythonhosted.org/packages/3f/c0/abde58b837e066bca19a3f7332d9d0493521d7dd6b48248451a9e3fe2214/h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl


In [ ]:
import tensorflow as tf
print(tf.__version__)
import keras
!pip install tensorflow-estimator==1.15.1

1.15.0


Using TensorFlow backend.


AttributeError: ignored

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [ ]:
## to solve ModuleNotFoundError: No module named 'keras.utils.training_utils'`
!pip uninstall keras 
!pip install keras==2.2.4

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import keras

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/drive/MyDrive/My Drive/Research/Thesis/Training/Project/main")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Path to Shapes trained weights
SHAPES_MODEL_PATH = os.path.join(ROOT_DIR, "/logs/object20210610T2313/mask_rcnn_object_0010.h5")

## Configurations

In [ ]:
import pycocotools.coco as coco
from mrcnn.config import Config
class InferenceConfig(Config):
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Notebook Preferences

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# Set weights file path
if config.NAME == "shapes":
    weights_path = SHAPES_MODEL_PATH
elif config.NAME == "COCO":
    weights_path = COCO_MODEL_PATH
# Or, uncomment to load the last model you trained
# weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

## Review Weight Stats

In [ ]:
# Show stats of all trainable weights    
visualize.display_weight_stats(model)

# Histograms of Weights

TODO: cleanup this part

In [ ]:
# Pick layer types to display
LAYER_TYPES = ['Conv2D', 'Dense', 'Conv2DTranspose']
# Get layers
layers = model.get_trainable_layers()
layers = list(filter(lambda l: l.__class__.__name__ in LAYER_TYPES, 
                layers))
# Display Histograms
fig, ax = plt.subplots(len(layers), 2, figsize=(10, 3*len(layers)),
                       gridspec_kw={"hspace":1})
for l, layer in enumerate(layers):
    weights = layer.get_weights()
    for w, weight in enumerate(weights):
        tensor = layer.weights[w]
        ax[l, w].set_title(tensor.name)
        _ = ax[l, w].hist(weight[w].flatten(), 50)
